In [5]:
import json, requests
import pandas as pd
import numpy as np
import io
import warnings
warnings.filterwarnings('ignore')

from joblib import dump, load

In [6]:
import matplotlib as mlab
import matplotlib.pyplot as plt
import seaborn as sns 
import time
import math
# import statsmodels.api as sm    
# import scipy.stats as stats 
# from pylab import rcParams

import sys
from functools import reduce
import itertools
import random

import datetime
from datetime import date, timedelta
from pandas.tseries.offsets import Day, MonthBegin, MonthEnd
# Libraries for stock data
import yfinance as yf

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
# from tensorflow.keras.layers import Conv1D,BatchNormalization, Dropout, Flatten, Input, Dense
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from tensorflow.keras.models import Sequential

2023-01-26 15:46:07.570669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 15:46:08.707661: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-26 15:46:08.707686: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-26 15:46:11.744722: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [7]:
import random
from random import sample
from transform_forecaster import stock_data_pull, fill_blanks, \
     change_shape, dataset, window_gen; 

Procedure for testing:
1. test model response with randomly generated data.
2. test model with data sourced from y finance (just a hand full of symbols)
3. include pre-process steps similar to those in direction model
4. Test model within .py file

In [9]:
#recon_model = keras.models.load_model('/home/cdsw/saved_models/model_01_17_23')
recon_model = keras.models.load_model('model_01_17_23')

**Random input data**

In [10]:
randomlist = []
for i in range(0,60):
  n = random.randint(95,130)
  randomlist.append(n)
# converting list to array
arr = np.array(randomlist)

test = arr.reshape(1,arr.shape[0],1)

pred = recon_model.predict(test)

1/1 [==============================] - 1s 790ms/step


In [11]:
pred

array([[ 99.96874 , 100.44134 , 100.07873 , 100.00119 ,  99.87107 ,
        100.055176,  99.73769 ,  99.4128  ,  99.27228 ,  99.07069 ,
         99.15012 ,  98.71578 ]], dtype=float32)

In [12]:
#!ls /home/cdsw/saved_models

**Load yfinance data**

In [13]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
tckr = table[0]
# convert to list
basket = tckr.Symbol.unique().tolist()

In [14]:
smp_basket = sample(basket,10)

In [15]:
# Pull stock data for given ticker symvols
stock_total = stock_data_pull(smp_basket,False);

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Name'], dtype='object')


In [16]:
stock_total.Datetime.drop_duplicates().sort_values()

0      2022-12-27 09:30:00-05:00
1      2022-12-27 09:35:00-05:00
2      2022-12-27 09:40:00-05:00
3      2022-12-27 09:45:00-05:00
4      2022-12-27 09:50:00-05:00
                  ...           
1572   2023-01-26 10:30:00-05:00
1573   2023-01-26 10:35:00-05:00
1574   2023-01-26 10:40:00-05:00
1575   2023-01-26 10:45:00-05:00
1576   2023-01-26 10:50:00-05:00
Name: Datetime, Length: 1577, dtype: datetime64[ns, America/New_York]

In [17]:
window = 60
date_range = stock_total.Datetime.drop_duplicates()
# added line below to clip off the lasest data
date_range = date_range.sort_values().tail(window)
date_range = pd.DataFrame(date_range)
stock_ready = fill_blanks(stock_total,'Name',date_range,'Datetime');

In [18]:
# date_range.sort_values('Datetime').tail(10)
# look at resultant dataframe
stock_ready.head()

stock_ready.dtypes

Datetime    datetime64[ns, America/New_York]
Close                                float64
Volume                               float64
Name                                  object
dtype: object

In [19]:
# check that all stocks have equal sizes
sizes = stock_ready.groupby('Name').size().unique()
sizes

array([60])

#### Generate predictions for actual stocks

In [20]:
stock_ready.Name.unique().shape

(10,)

In [21]:
symbol_list = []
for i, tckr in enumerate(stock_ready.Name.unique()):
    symbol_list.append(tckr)
    temp_input = stock_ready[stock_ready.Name == tckr].copy().Close.values
    if i == 0:
        input_array = temp_input
    else:
        input_array = np.append(input_array,temp_input)

In [22]:
input_array = input_array.reshape(i+1,window,1)

In [23]:
pred = recon_model.predict(input_array)

1/1 [==============================] - 1s 728ms/step


In [24]:
pred.shape

(10, 12)

In [25]:
i+1

10

In [26]:
#pred.tolist()
pred_time_list = [stock_ready.Datetime.max()]*(i+1)

In [27]:
tst_output = {'symbol':symbol_list,'pred_time':pred_time_list,'prediction':pred.tolist()}

Test .py file with forecating model

In [28]:
from fct_model_serve import forecast_price


Not running in a model replica, so using a local development
version of the model metrics service. Please use the following
CRN's to consume metrics:
   model_crn: "crn:cdp:ml:::workspace:dev/model" (cdsw.dev_model_crn)
   model_build_crn: "crn:cdp:ml:::workspace:dev/model-build" (cdsw.dev_model_build_crn)
   model_deployment_crn: "crn:cdp:ml:::workspace:dev/model-deployment" (cdsw.dev_model_deployment_crn)



#### Test deployed Model

In [29]:
# use this as input data
cols = ['symbol','symbol_time','close', 'volume']
stock_total.columns = ['symbol_time','close','volume','symbol']
stock_total.head()

,symbol_time,close,volume,symbol
0,2022-12-27 09:30:00-05:00,117.120003,196361,COP
1,2022-12-27 09:35:00-05:00,117.389999,69202,COP
2,2022-12-27 09:40:00-05:00,118.190002,49084,COP
3,2022-12-27 09:45:00-05:00,117.550003,44612,COP
4,2022-12-27 09:50:00-05:00,117.779999,57296,COP


In [43]:
model_url = 'https://modelservice.ml-5fea65b7-abd.loki-cdp.a465-9q4k.cloudera.site/model'
accessKey = 'mimv4t2o7vm39egwx3mml0wbog1sqrb9'


In [44]:
cols = stock_total.columns

In [45]:
# load test data
df = pd.read_csv('test_model_input.csv')

In [46]:
df.groupby('Datetime').size()
#df.head()

Datetime
2023-01-09 13:35:00    10
dtype: int64

In [47]:
stock_total.symbol_time = stock_total.symbol_time.astype("str") 

In [48]:
stock_total.dtypes

symbol_time     object
close          float64
volume           int64
symbol          object
dtype: object

In [49]:
# build input for api into dictionary w/ values as a list
input_data_mult = dict()
for col in cols:
    input_data_mult[col] = stock_total[col].to_list()    
    
# build embedded dictionary step 1    
request_dict = {"request":input_data_mult}

# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict

In [50]:
type(request_dict)

dict

In [51]:
# request_dict

In [54]:
r2 = requests.post(model_url, data=json.dumps(request_dict),headers={'Content-Type': 'application/json'})

In [55]:
r2.json()

{'success': True,
 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:7364d789-daae-47c6-951f-2ef3926b100c/ec1626fd-c2dc-4eb0-a2cc-12e526016936',
  'prediction': {'pred_time': ['2023-01-26 10:50:00-05:00',
    '2023-01-26 10:50:00-05:00',
    '2023-01-26 10:50:00-05:00',
    '2023-01-26 10:50:00-05:00',
    '2023-01-26 10:50:00-05:00',
    '2023-01-26 10:50:00-05:00',
    '2023-01-26 10:50:00-05:00',
    '2023-01-26 10:50:00-05:00',
    '2023-01-26 10:50:00-05:00',
    '2023-01-26 10:50:00-05:00'],
   'prediction': [[122.36091613769531,
     122.3556900024414,
     122.35311889648438,
     122.36701202392578,
     122.32926177978516,
     122.33344268798828,
     122.31700134277344,
     122.30119323730469,
     122.31839752197266,
     122.30509948730469,
     122.29951477050781,
     122.30147552490234],
    [150.6694793701172,
     150.66513061523438,
     150.67227172851562,
     150.71067810058594,
     150.6888885498047,
    

**Test using new payload syntax**<br>
this would simulate the way data comes out of CDP

In [73]:
model_url

'https://modelservice.ml-5fea65b7-abd.loki-cdp.a465-9q4k.cloudera.site/model'

In [56]:
accessKey = 'moz6mymy4dksvmj4ko97jxj5e5vqx38v'

In [62]:
stock_total['symbol'].unique()

array(['COP', 'WHR', 'TJX', 'DLR', 'MRK', 'IT', 'L', 'KHC', 'CFG', 'CDAY'],
      dtype=object)

In [64]:
test_input = stock_total[stock_total['symbol'] == 'TJX'].sort_values('symbol_time').tail(60)
test_input = test_input[cols]

In [65]:
# convert to expected payload
data_in = test_input.to_dict('records')

In [68]:
# build embedded dictionary step 1    
request_dict1 = {"data":data_in}

request_dict = {"request":request_dict1}


# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict

In [74]:
request_dict

{'accessKey': 'moz6mymy4dksvmj4ko97jxj5e5vqx38v',
 'request': {'data': [{'symbol_time': '2023-01-25 12:20:00-05:00',
    'close': 81.69619750976562,
    'volume': 16461,
    'symbol': 'TJX'},
   {'symbol_time': '2023-01-25 12:25:00-05:00',
    'close': 81.66999816894531,
    'volume': 43467,
    'symbol': 'TJX'},
   {'symbol_time': '2023-01-25 12:30:00-05:00',
    'close': 81.69999694824219,
    'volume': 30474,
    'symbol': 'TJX'},
   {'symbol_time': '2023-01-25 12:35:00-05:00',
    'close': 81.75,
    'volume': 40407,
    'symbol': 'TJX'},
   {'symbol_time': '2023-01-25 12:40:00-05:00',
    'close': 81.77999877929688,
    'volume': 22951,
    'symbol': 'TJX'},
   {'symbol_time': '2023-01-25 12:45:00-05:00',
    'close': 81.83000183105469,
    'volume': 16315,
    'symbol': 'TJX'},
   {'symbol_time': '2023-01-25 12:50:00-05:00',
    'close': 81.7699966430664,
    'volume': 18900,
    'symbol': 'TJX'},
   {'symbol_time': '2023-01-25 12:55:00-05:00',
    'close': 81.69999694824219,
   

In [71]:
 rtest = requests.post(model_url, data=json.dumps(request_dict),headers={'Content-Type': 'application/json'})

In [72]:
rtest.json()

{'success': True,
 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:7364d789-daae-47c6-951f-2ef3926b100c/6158dc37-a1d4-47d7-aeb6-cb712e537354',
  'prediction': {'pred_time': ['2023-01-26 10:45:00-05:00'],
   'prediction': [[82.08091735839844,
     82.07913208007812,
     82.07594299316406,
     82.08859252929688,
     82.07117462158203,
     82.08387756347656,
     82.07714080810547,
     82.07178497314453,
     82.0950927734375,
     82.08521270751953,
     82.08292388916016,
     82.08302307128906]],
   'symbol': ['TJX']},
  'uuid': '899a141e-8d8c-4b5f-a206-5dbb8a2c4ed1'},
 'ReplicaID': 'test-forecasting-model-2-11-46-b6c75c45c-klgkm',
 'Size': 548,
 'StatusCode': 200}

In [57]:
#input_data_mult

In [ ]:

# build embedded dictionary step 1    
request_dict = {"request":input_data_mult}

# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict